In [98]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import PolynomialFeatures

### Information of train and test datasets

In [99]:
train = pd.read_csv('train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   15000 non-null  int64  
 1   age                  15000 non-null  float64
 2   height(cm)           15000 non-null  float64
 3   weight(kg)           15000 non-null  float64
 4   waist(cm)            15000 non-null  float64
 5   eyesight(left)       15000 non-null  float64
 6   eyesight(right)      15000 non-null  float64
 7   hearing(left)        15000 non-null  float64
 8   hearing(right)       15000 non-null  float64
 9   systolic             15000 non-null  float64
 10  relaxation           15000 non-null  float64
 11  fasting blood sugar  15000 non-null  float64
 12  Cholesterol          15000 non-null  float64
 13  triglyceride         15000 non-null  float64
 14  HDL                  15000 non-null  float64
 15  LDL                  15000 non-null 

In [106]:
test = pd.read_csv('test.csv')


### Roc-auc score on ordinary dataset with DecisionTreeClassifier. 

In [101]:
X = train.drop('smoking', axis=1)
y = train['smoking']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = DecisionTreeClassifier(max_depth=6, min_impurity_decrease=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

acc = accuracy_score(y_test, y_pred)

y_score = model.predict_proba(X_test)[:, 1]
y_score_train = model.predict_proba(X_train)[:, 1]

roc = roc_auc_score(y_test, y_score)
roc_train = roc_auc_score(y_train, y_score_train)


print('ROC train', roc_train)
print('ROC AUC', roc)

ROC train 0.7612819902188186
ROC AUC 0.7504257003411299


## Feature engeneering

### Creating BMI column on train and test datasets

In [6]:
train['BMI'] = train['weight(kg)'] / (train['height(cm)']/100)**2
test['BMI'] = test['weight(kg)'] / (test['height(cm)']/100)**2


In [7]:
train = train.drop(columns=['weight(kg)', 'height(cm)'])
test = test.drop(columns=['weight(kg)', 'height(cm)'])

In [8]:
X = train.drop('smoking', axis=1)
y = train['smoking']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = DecisionTreeClassifier(max_depth=6, min_impurity_decrease=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

acc = accuracy_score(y_test, y_pred)

y_score = model.predict_proba(X_test)[:, 1]
y_score_train = model.predict_proba(X_train)[:, 1]

roc = roc_auc_score(y_test, y_score)
roc_train = roc_auc_score(y_train, y_score_train)


print('ROC train', roc_train)
print('ROC AUC', roc)

ROC train 0.7612819902188186
ROC AUC 0.7504257003411299


### Creating 

In [102]:
# 1. Calculate Body Mass Index (BMI)
train['BMI'] = train['weight(kg)'] / ((train['height(cm)'] / 100) ** 2)

# 2. Categorize Blood Pressure Levels
def categorize_bp(systolic, diastolic):
    if 110 <= systolic <= 120 and 70 <= diastolic <= 80:
        return 0
    elif 120 < systolic < 130 and diastolic < 80:
        return 1
    elif 130 <= systolic < 140 or 80 <= diastolic < 90:
        return 2
    elif systolic >= 140 or diastolic >= 90:
        return 3
    elif 110 > systolic or diastolic < 70:
        return -1
    else:
        return 'unknown'

train['BP_Category'] = train.apply(lambda row: categorize_bp(row['systolic'], row['relaxation']), axis=1)

# 3. Calculate Cholesterol Ratios
train['Cholesterol_HDL_Ratio'] = train['Cholesterol'] / train['HDL']
train['LDL_HDL_Ratio'] = train['LDL'] / train['HDL']

# 4. Calculate Waist-to-Height Ratio
train['Waist_Height_Ratio'] = train['waist(cm)'] / train['height(cm)']

# 5. Create Vision and Hearing Impairment Indicators
train['Vision_Impairment'] = ((train['eyesight(left)'] < 1) | (train['eyesight(right)'] < 1)).astype(int)
train['Hearing_Impairment'] = ((train['hearing(left)'] < 1) | (train['hearing(right)'] < 1)).astype(int)

# 6. Age Binning (example bins: Young, Middle-aged, Elderly)
train['Age_Group'] = pd.cut(train['age'], bins=[0, 35, 60, 100], labels=[0, 1, 2])

# 7. Combine Health Indicators to flag potential metabolic syndrome
train['Metabolic_Syndrome'] = ((train['fasting blood sugar'] > 100) &
                                    (train['Cholesterol'] > 200) &
                                    (train['Gtp'] > 40)).astype(int)

## 8. Flag High/Low Health Markers (example for hemoglobin, AST, and ALT)
#train['High_Hemoglobin'] = (train['hemoglobin'] > 15).astype(int)
#train['High_AST'] = (train['AST'] > 40).astype(int)
#train['High_ALT'] = (train['ALT'] > 40).astype(int)

# Check the newly created features
train.head()


,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,smoking,BMI,BP_Category,Cholesterol_HDL_Ratio,LDL_HDL_Ratio,Waist_Height_Ratio,Vision_Impairment,Hearing_Impairment,Age_Group,Metabolic_Syndrome
0,0,55.0,170.0,80.0,92.0,1.2,0.8,1.0,1.0,129.0,...,0.0,27.681661,1,3.571429,2.326531,0.541176,1,0,1,0
1,1,30.0,170.0,80.0,94.0,1.5,1.5,1.0,1.0,128.0,...,1.0,27.681661,2,3.686275,2.176471,0.552941,0,0,0,0
2,2,45.0,170.0,75.0,84.0,1.0,1.0,1.0,1.0,124.0,...,1.0,25.951557,2,3.692308,2.153846,0.494118,0,0,1,0
3,3,55.0,150.0,55.0,85.0,0.9,0.5,1.0,1.0,123.0,...,0.0,24.444444,1,3.163934,1.950820,0.566667,1,0,1,0
4,4,45.0,160.0,55.0,72.0,0.5,0.6,1.0,1.0,117.0,...,0.0,21.484375,0,3.229508,1.967213,0.450000,1,0,1,0


#### Feature engeneering for test dataset

In [107]:
# 1. Calculate Body Mass Index (BMI)
test['BMI'] = test['weight(kg)'] / ((test['height(cm)'] / 100) ** 2)

# 2. Categorize Blood Pressure Levels
def categorize_bp(systolic, diastolic):
    if 110 <= systolic <= 120 and 70 <= diastolic <= 80:
        return 0
    elif 120 < systolic < 130 and diastolic < 80:
        return 1
    elif 130 <= systolic < 140 or 80 <= diastolic < 90:
        return 2
    elif systolic >= 140 or diastolic >= 90:
        return 3
    elif 110 > systolic or diastolic < 70:
        return -1
    else:
        return 'unknown'

test['BP_Category'] = test.apply(lambda row: categorize_bp(row['systolic'], row['relaxation']), axis=1)

# 3. Calculate Cholesterol Ratios
test['Cholesterol_HDL_Ratio'] = test['Cholesterol'] / test['HDL']
test['LDL_HDL_Ratio'] = test['LDL'] / test['HDL']

# 4. Calculate Waist-to-Height Ratio
test['Waist_Height_Ratio'] = test['waist(cm)'] / test['height(cm)']

# 5. Create Vision and Hearing Impairment Indicators
test['Vision_Impairment'] = ((test['eyesight(left)'] < 1) | (test['eyesight(right)'] < 1)).astype(int)
test['Hearing_Impairment'] = ((test['hearing(left)'] < 1) | (test['hearing(right)'] < 1)).astype(int)

# 6. Age Binning (example bins: Young, Middle-aged, Elderly)
test['Age_Group'] = pd.cut(test['age'], bins=[0, 35, 60, 100], labels=[0, 1, 2])

# 7. Combine Health Indicators to flag potential metabolic syndrome
test['Metabolic_Syndrome'] = ((test['fasting blood sugar'] > 100) &
                                    (test['Cholesterol'] > 200) &
                                    (test['Gtp'] > 40)).astype(int)

## 8. Flag High/Low Health Markers (example for hemoglobin, AST, and ALT)
#test['High_Hemoglobin'] = (test['hemoglobin'] > 15).astype(int)
#test['High_AST'] = (test['AST'] > 40).astype(int)
#test['High_ALT'] = (test['ALT'] > 40).astype(int)

# Check the newly created features
test.head()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,dental caries,BMI,BP_Category,Cholesterol_HDL_Ratio,LDL_HDL_Ratio,Waist_Height_Ratio,Vision_Impairment,Hearing_Impairment,Age_Group,Metabolic_Syndrome
0,15000,40.0,175.0,70.0,84.0,1.5,1.5,1.0,1.0,120.0,...,0.0,22.857143,-1,3.977778,2.422222,0.480000,0,0,1,0
1,15001,45.0,155.0,55.0,72.4,0.6,0.5,1.0,1.0,102.0,...,0.0,22.892820,-1,3.321429,1.946429,0.467097,1,0,1,0
2,15002,40.0,160.0,55.0,76.0,1.2,1.2,1.0,1.0,115.0,...,0.0,21.484375,-1,2.168831,0.961039,0.475000,0,0,1,0
3,15003,45.0,150.0,50.0,74.4,1.0,1.0,1.0,1.0,96.0,...,0.0,22.222222,-1,2.105882,0.964706,0.496000,0,0,1,0
4,15004,35.0,185.0,80.0,90.0,0.8,1.0,1.0,1.0,113.0,...,0.0,23.374726,0,3.101695,1.779661,0.486486,1,0,0,0


In [108]:
train = train.drop(columns=['weight(kg)', 'height(cm)', 'Cholesterol', 'HDL', 'waist(cm)', 'height(cm)', 'eyesight(left)', 'eyesight(right)','hearing(left)', 'hearing(right)', 'age', 'fasting blood sugar', 'Cholesterol', 'Gtp', 'systolic', 'relaxation'])
test = test.drop(columns=['weight(kg)', 'height(cm)', 'Cholesterol', 'HDL', 'waist(cm)', 'height(cm)', 'eyesight(left)', 'eyesight(right)','hearing(left)', 'hearing(right)', 'age', 'fasting blood sugar', 'Cholesterol', 'Gtp', 'systolic', 'relaxation'])

In [109]:
test

,id,triglyceride,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,dental caries,BMI,BP_Category,Cholesterol_HDL_Ratio,LDL_HDL_Ratio,Waist_Height_Ratio,Vision_Impairment,Hearing_Impairment,Age_Group,Metabolic_Syndrome
0,15000,122.0,109.0,16.0,1.0,1.0,15.0,9.0,0.0,22.857143,-1,3.977778,2.422222,0.480000,0,0,1,0
1,15001,104.0,109.0,11.5,1.0,0.8,14.0,10.0,0.0,22.892820,-1,3.321429,1.946429,0.467097,1,0,1,0
2,15002,84.0,74.0,13.3,1.0,0.8,14.0,10.0,0.0,21.484375,-1,2.168831,0.961039,0.475000,0,0,1,0
3,15003,63.0,82.0,13.4,1.0,0.7,23.0,15.0,0.0,22.222222,-1,2.105882,0.964706,0.496000,0,0,1,0
4,15004,89.0,105.0,15.6,1.0,0.9,16.0,26.0,0.0,23.374726,0,3.101695,1.779661,0.486486,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,24995,50.0,108.0,14.9,1.0,1.0,23.0,16.0,0.0,24.221453,-1,2.966667,1.800000,0.500000,1,0,0,0
9996,24996,171.0,73.0,16.3,1.0,0.9,13.0,8.0,0.0,22.857143,2,3.000000,1.351852,0.485714,1,0,1,0
9997,24997,163.0,84.0,14.4,1.0,0.9,20.0,16.0,0.0,20.811655,-1,3.250000,1.615385,0.481290,1,0,1,0
9998,24998,52.0,118.0,12.2,1.0,0.8,17.0,9.0,0.0,21.484375,-1,2.380435,1.282609,0.462500,0,0,1,0


In [105]:
X = train.drop('smoking', axis=1)
y = train['smoking']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = DecisionTreeClassifier(max_depth=6, min_impurity_decrease=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

acc = accuracy_score(y_test, y_pred)

y_score = model.predict_proba(X_test)[:, 1]
y_score_train = model.predict_proba(X_train)[:, 1]

roc = roc_auc_score(y_test, y_score)
roc_train = roc_auc_score(y_train, y_score_train)


print('ROC train', roc_train)
print('ROC AUC', roc)

ROC train 0.7612819902188186
ROC AUC 0.7504257003411299


## Submission

In [110]:
# Get the predicted probabilities for the positive class (class 1)
y_prob = model.predict_proba(test)[:, 1]

# 12. Bashoratlarni saqlash
subm = pd.read_csv("sample_submission.csv")
subm['smoking'] = y_prob
subm.to_csv("my_submission_1.csv", index=False)